# Example 6502 Assembly Notebook

This is a quick guide to using 6502 Assembly in your Jupyter Notebooks.

The syntax we use here is that compatible with [`64tass`](https://tass64.sourceforge.net/).

If you want to find out more about the program used to emulate 6502 CPU execution, take a look at [this very interesting article](https://justine.lol/nesemu1.html) about Joel Yliluoma's emulator. Or marvel at the [source code](little_6502_emu.cc), which I have only lightly franken-patched to make work here.

## A Simple Example

Starting with something simple. Let's load `01` into the `A` register and store it in the `zeropage` address `$03`

In [10]:
LDA #$01  ; Load the value $01 to A
STA $03   ; Store it at $03

CPU Register Legend:                                   
C = Carry,    Z = Zero, I = Interrupt, D = Decimal Mode
V = Overflow, N = Negative                             
                                                       
Execution History (Values are after execution.)        
                                                       
                       | CPU Register | Program           
Instruction |  A  X  Y | C Z I D V N  | Counter           
----------- | -- -- -- | -----------  | -------           
LDA #0001   | 01 00 00 | 0 0 0 0 0 0  | 049154 
STA $0003   | 01 00 00 | 0 0 0 0 0 0  | 049156 
BRK         | 01 00 00 | 0 0 1 0 0 0  | 000000 
                                                       
First 256 Bytes of Zero Page After Execution Completed 
                                                       
    00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F    
    -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --    
00 |00 00 00 01 FF FF FF FF 00 00 00 00 FF FF FF FF 
01 |00 00 

### [What's all that about Ted?](https://www.youtube.com/watch?v=V_WUeqlZ5eE)

We get a lot of output! Let's break down what it all means.

The main meat is an execution history of our instructions:
```
                       | CPU Register | Program           
Instruction |  A  X  Y | C Z I D V N  | Counter           
----------- | -- -- -- | -----------  | -------           
LDA #0001   | 01 00 00 | 0 0 0 0 0 0  | 049154 
STA $0003   | 01 00 00 | 0 0 0 0 0 0  | 049156 
BRK         | 01 00 00 | 0 0 1 0 0 0  | 000000 
```

The first row shows our `LDA #$01` instruction being executed and the state of the registers afterwards. As you can see `01` has been loaded to the `A` register as expected:

```
Instruction |  A  X  Y | C Z I D V N  | Counter           
----------- | -- -- -- | -----------  | -------           
LDA #0001   | 01 00 00 | 0 0 0 0 0 0  | 049154 
```

Next, we store the value at address `$03` in the 'Zero Page` area of memory:

```
Instruction |  A  X  Y | C Z I D V N  | Counter           
----------- | -- -- -- | -----------  | -------           
STA $0003   | 01 00 00 | 0 0 0 0 0 0  | 049156 
```

When the assembly snippet finishes executing we get a dump of the first 256 bytes of zero-page memory. And in there we can see the `$01` we wrote to position `$03`:

```
First 256 Bytes of Zero Page After Execution Completed 
                                                       
    00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F    
    -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --    
00 |00 00 00 01 FF FF FF FF 00 00 00 00 FF FF FF FF 
```

So this exemplifies our basic usage pattern for 6502 ASM notebooks. We can use zero-page RAM as our 'output' and at the same time observe the state of the registers after the execution of each of our instructions. 

This hopefully makes Jupyter a useful REPL (Read, Eval, Print Loop) for prototyping 6502 assembly routines. 

Let's take a look at a more involved example.

## A More Involved Example

In this example we'll doing something a little more elaborate. We'll fill up the first 256 bytes of the zero page area with the value `$20`. Important stuff.

In [7]:
ZERO_PAGE = $00
;---------------------------------------
; A routine to fill the first 256 bytes
; of zero page with $20
;---------------------------------------
FillZeroPage
        LDX #$FF
        LDA #$20
_Loop   STA ZERO_PAGE,X
        DEX
        BNE _Loop
        RTS

; We call our routine
JSR FillZeroPage

CPU Register Legend:                                   
C = Carry,    Z = Zero, I = Interrupt, D = Decimal Mode
V = Overflow, N = Negative                             
                                                       
Execution History (Values are after execution.)        
                                                       
                       | CPU Register | Program           
Instruction |  A  X  Y | C Z I D V N  | Counter           
----------- | -- -- -- | -----------  | -------           
LDX #00FF   | 00 FF 00 | 0 0 0 0 0 1  | 049154 
LDA #0020   | 20 FF 00 | 0 0 0 0 0 0  | 049156 
STA $0000   | 20 FF 00 | 0 0 0 0 0 0  | 049158 
DEX         | 20 FE 00 | 0 0 0 0 0 1  | 049159 
BNE $00FB   | 20 FE 00 | 0 0 0 0 0 1  | 049156 
STA $0000   | 20 FE 00 | 0 0 0 0 0 1  | 049158 
DEX         | 20 FD 00 | 0 0 0 0 0 1  | 049159 
BNE $00FB   | 20 FD 00 | 0 0 0 0 0 1  | 049156 
STA $0000   | 20 FD 00 | 0 0 0 0 0 1  | 049158 
DEX         | 20 FC 00 | 0 0 0 0 0 1  | 049159 
BNE $00

# A Fibonacci Sequence

This snippet calculates a fibonacci number for you and stores the result in the Accumulator (`A`).

In [9]:
; Fibonacci calculator in 6502 asm
; by Pedro Franceschi (pedrohfranceschi@gmail.com)
; test it in http://www.6502asm.com/beta/index.html
; the accumulator in the end will hold the Nth fibonacci number
; https://gist.github.com/pedrofranceschi/1285964

LDX #$01  ; x = 1
STX $00   ; stores x

SEC       ; clean carry;

; Calculates 7th fibonacci number (13 = D in hex) 
; The result is stored in the Accumulator (A)
; (CHANGE HERE IF YOU WANT TO CALCULATE ANOTHER NUMBER)
LDY #$07


TYA       ; transfer y register to accumulator
SBC #$03  ; handles the algorithm iteration counting
TAY       ; transfer the accumulator to the y register
CLC       ; clean carry
LDA #$02  ; a = 2
STA $01   ; stores a

loop  LDX $01  ; x = a
      ADC $00  ; a += x
      STA $01  ; stores a
      STX $00  ; stores x
      DEY      ; y -= 1
      BNE loop ; jumps back to loop if Z bit != 0 
               ; (y's decremention isn't zero yet)


CPU Register Legend:                                   
C = Carry,    Z = Zero, I = Interrupt, D = Decimal Mode
V = Overflow, N = Negative                             
                                                       
Execution History (Values are after execution.)        
                                                       
                       | CPU Register | Program           
Instruction |  A  X  Y | C Z I D V N  | Counter           
----------- | -- -- -- | -----------  | -------           
LDX #0001   | 00 01 00 | 0 0 0 0 0 0  | 049154 
STX $0000   | 00 01 00 | 0 0 0 0 0 0  | 049156 
SEC         | 00 01 00 | 1 0 0 0 0 0  | 049157 
LDY #0007   | 00 01 07 | 1 0 0 0 0 0  | 049159 
TYA         | 07 01 07 | 1 0 0 0 0 0  | 049160 
SBC #0003   | 04 01 07 | 1 0 0 0 0 0  | 049162 
TAY         | 04 01 04 | 1 0 0 0 0 0  | 049163 
CLC         | 04 01 04 | 0 0 0 0 0 0  | 049164 
LDA #0002   | 02 01 04 | 0 0 0 0 0 0  | 049166 
STA $0001   | 02 01 04 | 0 0 0 0 0 0  | 049168 
LDX $00